In [65]:
import numpy as np
import pandas as pd
import json
import glob
from tqdm import tqdm

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('../data/processed/LNB_processed.xlsx')

In [71]:
stopwords = stopwords.words('german')

In [75]:
data = df.sample(200).full_text.values

In [6]:
nlp = spacy.load('../data/models/spacy_model_250421')

In [73]:
def lemmatize(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    texts_out = []
    
    for text in tqdm(texts):
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags and token.text.lower() not in stopwords:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    
    return texts_out

In [76]:
lemmatized_data = lemmatize(data)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:58<00:00,  1.12it/s]


In [77]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final

data_words = gen_words(lemmatized_data)

In [78]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

In [79]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')
                                        

In [80]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds', R=30)

In [81]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.370511 -0.124075       1        1  47.593524
14     0.279155 -0.246640       2        1  24.967509
17     0.115104 -0.334732       3        1  17.064306
10     0.141369  0.190834       4        1   2.549008
16    -0.001926 -0.172064       5        1   2.264580
8      0.114492  0.046888       6        1   1.353304
11    -0.025597  0.179664       7        1   1.045240
12    -0.155742 -0.031921       8        1   0.749802
4     -0.158160  0.058247       9        1   0.621653
0     -0.073354  0.104961      10        1   0.440191
2     -0.018049  0.032754      11        1   0.294221
7     -0.082383  0.064890      12        1   0.247097
6     -0.082982 -0.002431      13        1   0.238330
19    -0.084997  0.025925      14        1   0.185149
13    -0.058216  0.016918      15        1   0.154098
9     -0.063743  0.038744      16        1   0.093070
5     -0.047871  0.037172      17        1   0.091413
18    -0.054879  0.041865      18        1   0.042207
3     -0.056360  0.036495      19        1   0.002665
15    -0.056370  0.036506      20        1   0.002633, topic_info=         Term        Freq       Total Category  logprob  loglift
137      herr  456.000000  456.000000  Default  30.0000  30.0000
1632      rbl  216.000000  216.000000  Default  29.0000  29.0000
129       gut  478.000000  478.000000  Default  28.0000  28.0000
127      groß  672.000000  672.000000  Default  27.0000  27.0000
294    wasser  128.000000  128.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
157     leben    0.000117  269.938008  Topic20 -10.4337  -4.1076
494      haus    0.000117  249.342767  Topic20 -10.4345  -4.0291
40    bringen    0.000117  369.548285  Topic20 -10.4348  -4.4228
490    halten    0.000117  298.469934  Topic20 -10.4349  -4.2093
180    morgen    0.000117  156.146823  Topic20 -10.4349  -3.5615

[1186 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
6857       6  1.011372     abenteuer
6858       6  0.702114    abentheuer
771        1  0.842603   abgeordnete
771        2  0.093623   abgeordnete
771        7  0.046811   abgeordnete
...      ...       ...           ...
1299       2  0.984653  zuruckweisen
11222      5  0.803723          zwei
18711      4  0.983375         высот
18716      4  0.983389          году
18776      4  0.983388            сп

[1446 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 15, 18, 11, 17, 9, 12, 13, 5, 1, 3, 8, 7, 20, 14, 10, 6, 19, 4, 16])

In [54]:
for text in corpus:
    sums = 0
    for tup in text:
        sums += tup[1]
    print(sums/len(text))
        

1.2320185614849188
1.373913043478261
1.4423791821561338
1.2578849721706864
1.5482546201232033
1.0746268656716418
2.5480132450331126
1.3159784560143626
1.5822784810126582
1.2830188679245282
1.2035714285714285
1.3025210084033614
1.3543091655266757
1.1575342465753424
1.3006993006993006
1.1059322033898304
1.8482697426796806
1.401023890784983
1.154228855721393
1.1111111111111112
1.231818181818182
1.1380753138075315
1.2941176470588236
1.3083333333333333
2.18801652892562
1.749034749034749
1.3855721393034826
1.1770833333333333
1.2317880794701987
1.442696629213483
2.280672268907563
1.3328100470957613
1.4867109634551494
1.1965811965811965
1.4665981500513874
1.2136363636363636
1.8985148514851484
1.2505494505494505
1.270718232044199
1.3396226415094339
1.1486880466472302
1.3259423503325942
1.1111111111111112
1.2976190476190477
1.0929368029739777
1.6771826852531182
1.6252723311546842
1.6967895362663497
1.2695652173913043
1.16
1.5161691542288558
1.6064356435643565
1.3760245901639345
1.495738636363636

In [53]:
data[80]

'Der Nord-Ostsee-Canal und Vergleichung\ndesselben mit den Canälen von Suez,\nPanama und Korinth.\n\nlV\n! s\n, Jui Großen und Ganzen ist das Dahlströmsßodens\n,- sche Project als ein durchaus zweckmäßiged und wohl-\ndurchdachtes beibehalten, nur war, der erhöhten An-\n- spriiche der Kriegsmarine wegen und den beim Suezs\n« Canale inzwischen gemachten Erfahrungen entsprechend,\n- die Zahl der Krllmniungen zu ermäßigen, deren Halb-\ns messer von 2460 Fuß aus mindestens 3300 Fuß zu er-\n- höhen, sowie das Canalvrosil zu verbreitern und zu oers\n« tiefen. Jm Dahlström’schen Project waren fltr die west-\nk liche Einsahrt zwei Schleusen von beziehlich 413 Fuß\n" Länge bei 82 Fuß Breite und 252 Fuß Länge bei 41\ns Fuß Breite geplant, da diese aber fllr große Panier-\n« schiffe nicht genügen, so ist im Regierungdvroject noch\nj» eine Schleusewn 1180 Fuß Länge bei 197 Fuß Breite\nhinzugefügt, die im Stande sein wird nicht nur eine\ngrößere Anzahl von Kriegsschiffem sondern auch, zu\nZeiten auß

In [47]:
print(lemmatized_data[5])
print(data_words[5])

1. October Allerhöchsten Ukas 17. Sep tember Orianda baden S. Kaiser Allerhöchstdensclbcn gehörig Gut Orianda Südküste Krim I. Kaiserin verleihen ruhen Gut Orianda verbleiben Wille 11. Verwaltung laucht Graf Eollegien Koppen 19. Sept. Alupka Glück S. Großfürst Thronfolger S. überreicht plar Werk kostbar Brillantring empfangen schon erwähnen Dampfschiff Kaiser Fahrt und'wieder zurück heftig Sturm überstehen Hr. nun Glück haben S. Prinz höchst schmeichelhaft Schreiben fangen S. Erkenntlichkeit ausgezeichnet Führung Dampfschiffes erkennen geben Sehrei Geschenk Hrn. Brillant NamenSzoqe außerdem S. Bertheilung Mann schaft beilegen
['october', 'allerhochsten', 'ukas', 'sep', 'tember', 'orianda', 'baden', 'kaiser', 'gehorig', 'gut', 'orianda', 'sudkuste', 'krim', 'kaiserin', 'verleihen', 'ruhen', 'gut', 'orianda', 'verbleiben', 'wille', 'verwaltung', 'laucht', 'graf', 'eollegien', 'koppen', 'sept', 'alupka', 'gluck', 'großfurst', 'thronfolger', 'uberreicht', 'plar', 'werk', 'kostbar', 'brilla